In [1]:
import pandas as pd
import math
import numpy as np
import polars as pl


### Use this code to read and transform data if we begin with initial combined dataset


In [ ]:
chunksize = 100000  # Process 100,000 rows at a time
chunks = []

for chunk in pd.read_csv(
    r"C:\Users\user\Desktop\25 WI\Combined_WWLLN_Locations.txt", 
    delim_whitespace=True, 
    chunksize=chunksize
):
    chunks.append(chunk)

locations_WWLLN = pd.concat(chunks, ignore_index=True)


In [ ]:
track_file = pd.read_csv(r"C:\Users\user\Desktop\25 WI\Combined_Reduced_Trackfile.txt", sep="\t")
track_file = track_file.drop(track_file.columns[8], axis=1)
print(track_file.head(5))
# print(locations_WWLLN.head(5))
print(locations_WWLLN.shape)


In [ ]:
locations_WWLLN.columns = ['year', 'month', 'day', 'hour', 'min', 'sec','lat','lon','distance_from_storm_center_km_east', 'distance_from_storm_center_km_north', 'storm_code','storm_name']  # Replace with actual names
track_file.columns = ['year', 'month', 'day','hour','lat','lon','pressure', 'knots', 'storm_code', 'storm_name']

##### Calculate hypothenuse using existing north and east distance from the storm center and create inner core indicator - if 1 if hypotenuse is equal or less than 100km else 0 (rainband)

In [ ]:
locations_WWLLN['hypotenuse_disance_from_storm_center'] = np.sqrt(locations_WWLLN['distance_from_storm_center_km_east'] ** 2 +locations_WWLLN['distance_from_storm_center_km_north'] ** 2)
locations_WWLLN["inner_core_ind"] = locations_WWLLN["hypotenuse_disance_from_storm_center"].apply(
    lambda x: 1 if x <= 100 else 0
)

In [ ]:
print(track_file.shape)
print(locations_WWLLN.shape)
print(track_file[track_file['year'] == 2015].shape)

print(locations_WWLLN[locations_WWLLN['year'] == 2015].shape)
print(locations_WWLLN[locations_WWLLN['year'] == 2016].shape)
print(locations_WWLLN[locations_WWLLN['year'] == 2017].shape)
print(locations_WWLLN[locations_WWLLN['year'] == 2018].shape)
print(locations_WWLLN[locations_WWLLN['year'] == 2019].shape)
print(locations_WWLLN[locations_WWLLN['year'] == 2020].shape)

# print(locations_WWLLN[(locations_WWLLN['year'] == 2016) & (locations_WWLLN['month'] == 12)].shape)

# Re-check the shape after the transformation
print(track_file.head())
print(locations_WWLLN.head())


In [24]:
#track_file_2015.to_csv('track_file_2015_2015.csv', index=False)  
#locations_WWLLN_2015.to_csv('Combined_WWLLN_Locations_2015.csv', index=False)  

In [ ]:
track_file_pl = pl.from_pandas(track_file)
locations_WWLLN_pl = pl.from_pandas(locations_WWLLN)

In [27]:
track_file.to_csv('Combined_Reduced_Trackfile_w_header.csv', index=False)  
locations_WWLLN_pl.write_csv('Combined_WWLLN_Locations_w_header.txt')


##### Exploratory Data Analysis
1) Number of unique storm per year in WWLLN and trackfile
2) Number of lightning for each year & storm code
3) Number of inner core lightning (3% of total)

In [ ]:
locations_WWLLN.groupby('year')['storm_code'].nunique()
track_file.groupby('year')['storm_code'].nunique()

In [ ]:
testing = locations_WWLLN.groupby(['year', 'storm_code']).size()
testing = testing.reset_index(name='count')
print(testing)

In [4]:
print(locations_WWLLN[(locations_WWLLN['inner_core_ind'] == 1)].shape)

NameError: name 'locations_WWLLN' is not defined

#### Use this if we direclty read trasformed data set in polar format - which is in the google drive

In [ ]:
locations_WWLLN_pl = pl.read_csv('Combined_WWLLN_Locations_w_header.txt', separator=" ")  # For tab-separated values
track_file_pl = pl.read_csv('Combined_Reduced_Trackfile_w_header.csv', separator=" ")  # For tab-separated values

##### Function that joins two data set ...
My kernel broke here btw

In [ ]:
combined_dates = locations_WWLLN_pl.join_asof(
    track_file_pl,
    on="hour",
    by=["year", "month", "day", "storm_code", "storm_name"],  # Optional: match on multiple keys
    strategy="backward"  # "backward" (default) or "forward"
)

In [ ]:
print(combined_dates.shape)